In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
import tifffile

In [7]:
pip install imagecodecs


   ---------------------------------------- 0.0/25.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.4 MB 660.6 kB/s eta 0:00:39
   ---------------------------------------- 0.1/25.4 MB 550.5 kB/s eta 0:00:47
   ---------------------------------------- 0.1/25.4 MB 585.1 kB/s eta 0:00:44
   ---------------------------------------- 0.2/25.4 MB 821.4 kB/s eta 0:00:31
   ---------------------------------------- 0.2/25.4 MB 787.7 kB/s eta 0:00:33
   ---------------------------------------- 0.3/25.4 MB 983.9 kB/s eta 0:00:26
    --------------------------------------- 0.4/25.4 MB 1.2 MB/s eta 0:00:22
    --------------------------------------- 0.5/25.4 MB 1.4 MB/s eta 0:00:18
    --------------------------------------- 0.6/25.4 MB 1.5 MB/s eta 0:00:17
   - -------------------------------------- 0.8/25.4 MB 1.6 MB/s eta 0:00:16
   - -------------------------------------- 0.9/25.4 MB 1.7 MB/s eta 0:00:15
 

In [11]:
from PIL import Image


def load_image(file_path):
    # Load the image using PIL
    image = Image.open(file_path)

    # Convert the image to grayscale
    image = image.convert('L')

    # Transform the image to tensor
    transform = transforms.ToTensor()
    image_tensor = transform(image)

    # Add batch dimension
    image_tensor = image_tensor.unsqueeze(0)

    return image_tensor


# Example usage:
file_path = r"D:\College files\TISS\Data\Sentinal.tif"
image = load_image(file_path)
height, width = image.shape[2], image.shape[3]  # Get image height and width
model = UNet()
output = model(image)
print(output.size())

More samples per pixel than can be decoded: 23


UnidentifiedImageError: cannot identify image file 'D:\\College files\\TISS\\Data\\Sentinal.tif'

In [8]:

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        self.max_pool_2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.down_conv_1 = DoubleConv(1, 64)
        self.down_conv_2 = DoubleConv(64, 128)
        self.down_conv_3 = DoubleConv(128, 256)
        self.down_conv_4 = DoubleConv(256, 512)
        self.down_conv_5 = DoubleConv(512, 1024)

        self.up_trans_1 = nn.ConvTranspose2d(
            1024, 512, kernel_size=2, stride=2)
        self.up_conv_1 = DoubleConv(1024, 512)

        self.up_trans_2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.up_conv_2 = DoubleConv(512, 256)

        self.up_trans_3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.up_conv_3 = DoubleConv(256, 128)

        self.up_trans_4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.up_conv_4 = DoubleConv(128, 64)

        self.out = nn.Conv2d(64, 2, kernel_size=1)

    def forward(self, x):
        # Encoder
        x1 = self.down_conv_1(x)
        x2 = self.max_pool_2x2(x1)
        x3 = self.down_conv_2(x2)
        x4 = self.max_pool_2x2(x3)
        x5 = self.down_conv_3(x4)
        x6 = self.max_pool_2x2(x5)
        x7 = self.down_conv_4(x6)
        x8 = self.max_pool_2x2(x7)
        x9 = self.down_conv_5(x8)

        # Decoder
        x = self.up_trans_1(x9)
        x = F.interpolate(x, size=x7.size()[
                          2:], mode='bilinear', align_corners=True)
        x = self.up_conv_1(torch.cat([x, x7], 1))

        x = self.up_trans_2(x)
        x = F.interpolate(x, size=x5.size()[
                          2:], mode='bilinear', align_corners=True)
        x = self.up_conv_2(torch.cat([x, x5], 1))

        x = self.up_trans_3(x)
        x = F.interpolate(x, size=x3.size()[
                          2:], mode='bilinear', align_corners=True)
        x = self.up_conv_3(torch.cat([x, x3], 1))

        x = self.up_trans_4(x)
        x = F.interpolate(x, size=x1.size()[
                          2:], mode='bilinear', align_corners=True)
        x = self.up_conv_4(torch.cat([x, x1], 1))

        x = self.out(x)
        return x


def load_image(file_path):
    # Load the image using tifffile
    image = tifffile.imread(file_path)

    # Convert the image to grayscale
    # Assuming it's a single-channel image
    image = image[np.newaxis, ...]  # Add batch dimension

    # Transform the image to tensor
    image_tensor = torch.from_numpy(image).float()

    return image_tensor


# Example usage:
# Update with your image file path
file_path = "D:\College files\TISS\Data\Sentinal.tif"
image = load_image(file_path)
model = UNet()
output = model(image)
print(output.size())

<>:94: SyntaxWarning: invalid escape sequence '\C'
<>:94: SyntaxWarning: invalid escape sequence '\C'
C:\Users\HP\AppData\Local\Temp\ipykernel_17176\3325350476.py:94: SyntaxWarning: invalid escape sequence '\C'
  file_path = "D:\College files\TISS\Data\Sentinal.tif"


ValueError: <COMPRESSION.LZW: 5> requires the 'imagecodecs' package